In [1]:
import numpy as np
import array

# Example 2D numpy array
numpy_array = np.array([[1, 2, 3], [4, 5, 6]])

# Flatten the array
flattened_array = numpy_array.flatten()

# Convert to Python array - assuming the numpy array is of type int
python_array = array.array('i', flattened_array)

# Now you can pass 'python_array' to your C function
print(python_array)

array('i', [1, 2, 3, 4, 5, 6])


In [ ]:
from time import time as now
from array import array
import math

import metalcompute as mc

# threads_per_threadgroup seems to be 1024.
kernel = """
#include <metal_stdlib>
using namespace metal;

kernel void test(const device float *in,// [[ buffer(0) ]],
                const device float* in2 [[ buffer(1) ]],
                device float  *out [[ buffer(2) ]],
                uint lid [[ thread_position_in_threadgroup ]],
                uint gid [[ threadgroup_position_in_grid ]],
                uint num_threads [[threads_per_threadgroup]]
                ) {
    int i = gid * num_threads + lid;
    float value = in[i];
    float fixed = in2[0];
    out[i] = sin(value) + fixed;
}
"""

dev = mc.Device()
print(dev)

# count = 7234567
# arr = range(count)
arr = list(range(1000)) * 100000
count = len(arr)
in_buf = array('f',arr) # Can use as-is for input
constant = array('f',[1.0]) # Can use as-is for input
out_buf = dev.buffer(count*4)
out_buf_mv = memoryview(out_buf).cast('f')

function_name = "test"

# This should work
fn_good = dev.kernel(kernel).function(function_name)

print("Calculating sin of",count,"values")

s1 = now()
# This should work. Arrays must be 1D float at the moment
fn_good(count, in_buf, constant, out_buf)
e1 = now()


s2 = now()
oref = array('f',[math.sin(value)+1.0 for value in in_buf])
e2 = now()

print("Expected value:",oref[0], "Received value:",out_buf_mv[0])
print("Expected value:",oref[1], "Received value:",out_buf_mv[1])
print("Expected value:",oref[2], "Received value:",out_buf_mv[2])
print("Expected value:",oref[-1], "Received value:",out_buf_mv[-1])
print("Expected value:",oref[-2], "Received value:",out_buf_mv[-2])
print("Expected value:",oref[-3], "Received value:",out_buf_mv[-3])
# same = 0
# for i in range(count):
#     if -0.001 < oref[i] - out_buf_mv[i] < 0.001:
#         # print(i)
#         same +=1 
#         pass
print("same", same)
print("Metal compute took:",e1-s1,"s")
print("Reference compute took:",e2-s2,"s")

metalcompute.Device(Apple M2)
Calculating sin of 100000000 values
Expected value: 1.0 Received value: 1.0
Expected value: 1.8414709568023682 Received value: 1.8414709568023682
Expected value: 1.9092974662780762 Received value: 1.9092974662780762
Expected value: 0.9735392332077026 Received value: 0.9735419154167175
Expected value: 0.14452685415744781 Received value: 0.14449740946292877
Expected value: 0.10203251987695694 Received value: 0.10204171389341354


In [3]:
# let's do a matmul.

